# Imports

In [10]:
from enum import StrEnum
from typing import Annotated
import lancedb
from openai import AsyncOpenAI
import pandas as pd
from pydantic import BaseModel, Field
from sentence_transformers import SentenceTransformer

# Retrieval Augmented Generation (RAG)

[Function Calling](https://platform.openai.com/docs/guides/function-calling?api-mode=chat)

In [9]:
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cpu",
)
db = lancedb.connect("./data/lance_db")
tbl = db.open_table("movies")

In [ ]:
def query(text: str, limit: int = 10) -> pd.DataFrame:
    q_emb = embedder.encode(text)
    res = tbl.search(q_emb, limit=limit).to_pandas()

# Sentiment Analysis

[Structured Output](https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat)

In [ ]:
class Polarity(StrEnum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"


class SentimentAnalysisOutput(BaseModel):
    polarity: Annotated[Polarity, "The sentiment polarity of the text"]
    confidence: Annotated[
        float,
        Field(
            description="The confidence score of the sentiment polarity between 0 and 1",
            ge=0.0,
            le=1.0,
        ),
    ]

In [ ]:
SentimentAnalysisOutput(polarity="positive", confidence=1.1)

ValidationError: 1 validation error for SentimentAnalysisOutput
confidence
  Input should be less than or equal to 1 [type=less_than_equal, input_value=1.1, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal